In [ ]:
from math import tau
import numpy as np
import polars as pl

from rtsim import Axis, Body, Frame, Mount, Testbed, World, ConstantPVA, TimePVA

import matplotlib.pyplot as plt
plt.rc('text', usetex=True)
plt.rc('text.latex',
       preamble='\n'.join([r'\usepackage{siunitx}',
                           r'\usepackage{amsmath}',
                           r'\DeclareSIUnit{\gravity}{\textsl{g}}']))
plt.rc('font', **{'family': 'garamond', 'size': 8})
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8


In [ ]:
def wrap360(angle: float) -> float:
    """
    Wrap angle around 360 degrees.

    :param angle: Angle to wrap, degrees
    :type angle: float
    :return: Wrapped angle, degrees
    :rtype: float
    """
    return ((angle % 360) + 360) % 360


In [ ]:
def create_axis(amp: float, ws: int, wd: int, time: np.ndarray, starts: list[int]) -> tuple:
    """
    Create an axis dataset.

    :param amp: Amplitude of the angular acceleration pulse, degree/s/s.
    :type amp: float
    :param ws: Width of the angular rate pulse, ms.
    :type ws: int
    :param wd: Width of the angular acceleration pulse, ms.
    :type wd: int
    :param time: Time array of data.
    :type time: np.ndarray
    :param starts: Start times of each angular rate pulse, ms.
    :type starts: list[int]
    :return: Tuple of angular positions, rates, and accelerations.
    :rtype: tuple
    """
    theta = np.zeros_like(time)
    omega = np.zeros_like(time)
    omdot = np.zeros_like(time)

    for start in starts:
        i1 = np.where((time >= start) & (time <= start+wd))[0]
        i2 = np.where((time >= start+ws) & (time <= start+ws+wd))[0]
        i3 = np.where((time >= start-1) & (time <= start+ws+wd+1))[0]

        omdot[i1] = dh
        omdot[i2] = -dh
        omega[i3] = np.cumsum(omdot[i3]) * 1e-3
        theta[i3] = wrap360(np.cumsum(omega[i3]) * 1e-3)

    return theta, omega, omdot


In [ ]:
dh = 100.
ws = 72000
wd = 600

time = np.arange(0, 600001)

ti, wi, di = create_axis(dh, ws, wd, time, [ 14_723, 249_423, 328_479, 484_054])
tm, wm, dm = create_axis(dh, ws, wd, time, [ 93_115, 249_423, 406_311, 484_054])
to, wo, do = create_axis(dh, ws, wd, time, [171_200, 328_479, 406_311, 484_054])

data = pl.DataFrame(
    {
        'time': time,
        'ti': ti * tau / 360,
        'tm': tm * tau / 360,
        'to': to * tau / 360,
        'wi': wi * tau / 360,
        'wm': wm * tau / 360,
        'wo': wo * tau / 360,
        'di': di * tau / 360,
        'dm': dm * tau / 360,
        'do': do * tau / 360,
    }
)


In [ ]:
world = World('Terra', 6378137.0, 298.257223563, 7.2921150E-5)

In [ ]:
outer = Axis(
    "Outer",
    Frame(
        linear=ConstantPVA(
            p=np.zeros((3, 1)),
            v=np.zeros((3, 1)),
            a=np.zeros((3, 1)),
        ),
        angular=ConstantPVA(
            p=np.array([[0.], [0.], [0.]]),
            v=np.zeros((3, 1)),
            a=np.zeros((3, 1)),
        )
    )
)


In [ ]:
middle = Axis(
    "Middle",
    Frame(
        linear=ConstantPVA(
            p=np.zeros((3, 1)),
            v=np.zeros((3, 1)),
            a=np.zeros((3, 1)),
        ),
        angular=ConstantPVA(
            p=np.array([[0.], [tau/4], [0.]]),
            v=np.zeros((3, 1)),
            a=np.zeros((3, 1)),
        )
    )
)


In [ ]:
inner = Axis(
    "Inner",
    Frame(
        linear=ConstantPVA(
            p=np.array([[-0.1524], [0.], [0.]]),
            v=np.zeros((3, 1)),
            a=np.zeros((3, 1)),
        ),
        angular=ConstantPVA(
            p=np.array([[0.], [tau/4], [0.]]),
            v=np.zeros((3, 1)),
            a=np.zeros((3, 1)),
        )
    )
)


In [ ]:
mount = Mount(
    "Mount",
    Frame(
        linear=ConstantPVA(
            p=np.array([[0.3284], [0.], [0.]]),
            v=np.zeros((3, 1)),
            a=np.zeros((3, 1)),
        ),
        angular=ConstantPVA(
            p=np.array([[0.], [-tau/4], [0.]]),
            v=np.zeros((3, 1)),
            a=np.zeros((3, 1)),
        )
    )
)


In [ ]:
body = Body(
    "UUT",
    Frame(
        linear=ConstantPVA(
            p=np.array([[0.0164], [0.], [0.]]),
            v=np.zeros((3, 1)),
            a=np.zeros((3, 1)),
        ),
        angular=ConstantPVA(
            p=np.array([[tau/4], [0.], [0.]]),
            v=np.zeros((3, 1)),
            a=np.zeros((3, 1)),
        )
    )
)


In [ ]:
testbed = Testbed(
    "TART",
    llhg=(35.051339, -106.545044, 1630., 9.7920631997),
    components=(world, [inner, middle, outer], mount, body)
)


In [ ]:
misalignments = [Frame(
    linear=TimePVA(
        p=np.zeros((data.shape[0], 3, 1)),
        v=np.zeros((data.shape[0], 3, 1)),
        a=np.zeros((data.shape[0], 3, 1)),
    ),
    angular=TimePVA(
        p=np.zeros((data.shape[0], 3, 1)),
        v=np.zeros((data.shape[0], 3, 1)),
        a=np.zeros((data.shape[0], 3, 1)),
    )
)]*3


In [ ]:
rotations = []
for axis in ["i", "m", "o"]:
    theta = np.zeros((data.shape[0], 3, 1))
    omega = np.zeros((data.shape[0], 3, 1))
    omega_dot = np.zeros((data.shape[0], 3, 1))

    theta[:, 2, 0] = data[f"t{axis}"].to_numpy()
    omega[:, 2, 0] = data[f"w{axis}"].to_numpy()
    omega_dot[:, 2, 0] = data[f"d{axis}"].to_numpy()

    rotations.append(
        Frame(
            linear=ConstantPVA(
                p=np.zeros((3, 1)),
                v=np.zeros((3, 1)),
                a=np.zeros((3, 1)),
            ),
            angular=TimePVA(
                p=theta,
                v=omega,
                a=omega_dot,
            )
        )
    )


In [ ]:
testbed.process(
    time=data['time'].to_numpy(),
    misalignments=misalignments,
    rotations=rotations,
)


In [ ]:
result = pl.DataFrame(
    {
        'time': testbed.time,
        'fx': testbed.sfbib[:, 0, 0],
        'fy': testbed.sfbib[:, 1, 0],
        'fz': testbed.sfbib[:, 2, 0],
        'wx': testbed.avbib[:, 0, 0],
        'wy': testbed.avbib[:, 1, 0],
        'wz': testbed.avbib[:, 2, 0],
    }
)


In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(7, 6))
axs[0].plot(result["time"]/1e3, result["fx"]/9.80665)
axs[1].plot(result["time"]/1e3, result["fy"]/9.80665)
axs[2].plot(result["time"]/1e3, result["fz"]/9.80665)

for ax, axi in zip(axs[:2], ["x", "y"], strict=True):
    ax.set(ylabel=fr"$f_{axi},\ \unit{{\gravity}}$", ylim=[-1.2, 1.2], xticklabels=[])
    ax.grid(alpha=0.2)
    ax.spines[:].set_visible(False)

axs[1].spines["top"].set_visible(True)
axs[2].set(xlabel=r'Elapsed Time, \unit{\second}', ylabel=r"$f_z,\ \unit{{\gravity}}$", ylim=[-1.2, 1.2])
axs[2].grid(alpha=0.2)
axs[2].spines[["left", "right", "bottom"]].set_visible(False)

fig.subplots_adjust(wspace=0, hspace=0)


In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(7, 6))
axs[0].plot(result["time"]/1e3, result["wx"])
axs[1].plot(result["time"]/1e3, result["wy"])
axs[2].plot(result["time"]/1e3, result["wz"])

for ax, axi in zip(axs[:2], ["x", "y"], strict=True):
    ax.set(ylabel=fr"$\omega_{axi},\ \unit{{\radian/\second}}$", ylim=[-1.2, 1.2], xticklabels=[])
    ax.grid(alpha=0.2)
    ax.spines[:].set_visible(False)

axs[1].spines["top"].set_visible(True)
axs[2].set(xlabel=r'Elapsed Time, \unit{\second}', ylabel=r"$\omega_z,\ \unit{{\radian/\second}}$", ylim=[-1.2, 1.2])
axs[2].grid(alpha=0.2)
axs[2].spines[["left", "right", "bottom"]].set_visible(False)

fig.subplots_adjust(wspace=0, hspace=0)
